In [4]:
from transformers import pipeline
import pandas as pd

In [5]:
def load_model(path_to_folder_with_model, device=None):
  model = pipeline('text-classification',
                   model=path_to_folder_with_model,
                   tokenizer='DeepPavlov/rubert-base-cased-conversational',
                   framework='pt',
                   device=device)
  return model

def predict(model, feedback):
  return model(feedback[1] + ' ' + feedback[3])

In [22]:
pos_model = load_model('/content/drive/MyDrive/Hakaton/Models_for_Kolbasa/pos_neg_model')
obj_model = load_model('/content/drive/MyDrive/Hakaton/Models_for_Kolbasa/object_model')
rel_model = load_model('/content/drive/MyDrive/Hakaton/Models_for_Kolbasa/relevant_model')

pos_model('Уууу, курс плохой, не буду на него ходить')

[{'label': 'negative', 'score': 0.870173454284668}]

In [18]:
test = pd.read_csv('/content/drive/MyDrive/Hakaton/test_data.csv')
test['is_positive'] = [int(d['label'] == 'positive') for d in pos_model((test['question_2'] + test['question_4']).to_list())]
test['is_relevant'] = [int(d['label'] == 'relevant') for d in rel_model((test['question_2'] + test['question_4']).to_list())]
test['object'] = [{'webinar': 0, 'program': 1, 'tutor': 2}[d['label']] for d in obj_model((test['question_2'] + test['question_4']).to_list())]

In [21]:
test[['hash', 'is_relevant', 'object', 'is_positive']].to_csv('/content/submission.csv', index=False)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Hakaton/train_data.csv').iloc[:, 1:-3]
sample = df.iloc[1].to_list()
sample

['Основы программирования',
 'Подробное объяснение логических операторов в JavaScript помогло мне лучше понять условные конструкции.',
 'Нет.',
 'Предоставить ещё больше практических задач после теории.',
 'Детальное изучение асинхронного программирования в JavaScript.']

# Код алгоса, который раскидывает баллы

In [ ]:
from difflib import get_close_matches

pos_model = load_model('/content/drive/MyDrive/Hakaton/Models_for_Kolbasa/pos_neg_model')
obj_model = load_model('/content/drive/MyDrive/Hakaton/Models_for_Kolbasa/object_model')
rel_model = load_model('/content/drive/MyDrive/Hakaton/Models_for_Kolbasa/relevant_model')

def words_in_sentence(words, sentence):
  for word in words:
    if get_close_matches(word, sentence.split()):
      return 1
  return 0

def get_ind(feedback):

  out = {'metodist_positive_ind_1_present': 0,
       'metodist_positive_ind_2_knowledgePractice': 0,
       'metodist_positive_ind_3_knowledge': 0,
       'professor_positive__ind_1_speach': 0,
       'professor_positive__ind_2_material': 0,
       'professor_positive__ind_3_communication': 0,
       'metodist_negative_ind_1_badExamples': 0,
       'metodist_negative_ind_2_badMaterial': 0,
       'metodist_negative_ind_3_badKnowledge': 0,
       'professor_negative__ind_1_badspeach': 0,
       'professor_negative__ind_2_badMaterial': 0,
       'professor_negative__ind_3_badcommunication': 0,
       'is_critical': 0}

  words = ['проблемы', 'подключение', 'ужасно', 'отвратительный', 'игнорируют', 'не отвечают', 'поддержка']
  if words_in_sentence(words, '  '.join(feedback)):
    out['is_critical'] = 1
    return out

  if rel_model(feedback)[0]['label'] == 'not_relevant':
    return out

  positive = pos_model(feedback)[0]['label'] == 'positive'
  obj = obj_model(feedback)[0]['label']

  if positive:
    if obj == 'webinar':
      words = ['увлекательно', 'простой', 'понятный', 'спасибо']
      out['metodist_positive_ind_1_present'] = words_in_sentence(words, feedback[1])
      words = ['карьера', 'работа', 'собеседование', 'будущее']
      out['metodist_positive_ind_2_knowledgePractice'] = max(words_in_sentence(['нет'], feedback[2]),
                                                             words_in_sentence(words, feedback[1]))
      words = ['актуальный', 'современный', 'полезный']
      out['metodist_positive_ind_3_knowledge'] = words_in_sentence(words, feedback[1])
    elif obj == 'tutor':
      words = ['впечатляющий', 'интересный', 'захватывающий', 'познавательный', 'увлекательно', 'простой', 'понятный', 'спасибо']
      out['professor_positive__ind_1_speach'] = words_in_sentence(words, feedback[1])
      words = ['отвечает', 'помогает', 'помощь', 'лучший', 'купите мне шаурму']
      out['professor_positive__ind_3_communication'] = max(words_in_sentence(['нет'], feedback[2]),
                                                           words_in_sentence(words, feedback[1]))
      words = ['актуальный', 'современный', 'полезный']
      out['professor_positive__ind_2_material'] = words_in_sentence(words, feedback[1])
  else:
    if obj == 'webinar':
      words = ['нет структуры', 'непонятно', 'плохо', 'не понял']
      out['metodist_negative_ind_1_badExamples'] = words_in_sentence(words, feedback[2] + ' ' + feedback[3])
      words = ['неактуально', 'неинтересно', 'фигня']
      out['metodist_negative_ind_2_badMaterial'] = words_in_sentence(words, feedback[2] + ' ' + feedback[3])
      words = ['устаревший', 'неактуальный', 'плохой']
      out['metodist_negative_ind_3_badKnowledge'] = words_in_sentence(words, feedback[2] + ' ' + feedback[3])
    elif obj == 'tutor':
      words = ['скучный', 'трудности', 'сложный', 'монотонный', 'запутанный', 'мало']
      out['professor_negative__ind_1_badspeach'] = words_in_sentence(words, feedback[2] + ' ' + feedback[3])
      words = ['плохо', 'грубый']
      out['professor_negative__ind_3_badcommunication'] = words_in_sentence(words, feedback[2] + ' ' + feedback[3])
      words = ['не знает', 'неактуально', 'не разбирается']
      out['professor_negative__ind_2_badMaterial'] = words_in_sentence(words, feedback[2] + ' ' + feedback[3])
  return out

In [ ]:
get_ind(sample)

{'metodist_positive_ind_1_present': 0,
 'metodist_positive_ind_2_knowledgePractice': 0,
 'metodist_positive_ind_3_knowledge': 0,
 'professor_positive__ind_1_speach': 1,
 'professor_positive__ind_2_material': 0,
 'professor_positive__ind_3_communication': 1,
 'metodist_negative_ind_1_badExamples': 0,
 'metodist_negative_ind_2_badMaterial': 0,
 'metodist_negative_ind_3_badKnowledge': 0,
 'professor_negative__ind_1_badspeach': 0,
 'professor_negative__ind_2_badMaterial': 0,
 'professor_negative__ind_3_badcommunication': 0}